In [108]:
import pandas as pd
from pathlib import Path
from dataclasses import dataclass, field
import re


In [133]:
cur_dir = Path.cwd()
data_dir = cur_dir / 'data'
if not data_dir.exists:
    data_dir.mkdir()


@dataclass
class Category:
    name: str
    key_words: list[str] = field(default_factory=list)
    transaction: pd.Series = field(default_factory=lambda : pd.Series)
    bank_category: list[str] = field(default_factory=list)


In [177]:
food = Category(
    name='Продукты',
    key_words=['Albert', 'Lidl', 'Kaufland', 'Billa', 'ASIAN MANGO', 'maso',
               'GLOBUS BRNO', 'Potraviny FOLKOVA'])
food_out_side = Category(
    name='Еда на вынос',
    key_words=['VENTANA', 'MOTMOT', 'McDonalds', 'KFC', 'Burger King',
               'Restaurant', 'Restaurace', 'CHILLI TREE', 'KOFIBOX','SUSHIMIX',
               'Damejidlo', 'Pizza', 'NEEXISTUJE', 'KAFE', 'JIDELNI VUZ']
    )
go_pay = Category(
    name='GoPay',
    key_words=['GoPay'])
additional = Category(
    name='Сторонние покупки',
    key_words=['DATART', 'PlayStation', 'CESKA POSTA', 'CINEMA', 'KVETINY',
               'CELIO', 'PRIMARK', 'SB Olympia Brno Brno', 'Kino', 'ROSSMANN',
               'HULK GYM', 'HULK GYM', 'relaxin'
               ])
regular_payments = Category(
    name='Регулярные платежи',
    key_words=['hetzner', 'SPOTIFY', 'NETFLIX', 'BARBER', 'BUBELINY', 'Vodafone'],
    bank_category=['Nájem'])
transport = Category(
    name='Транспорт',
    key_words=['DPP', 'Flixbus', 'www.cd.cz', 'MPLA', 'EDALNICE', 'IDSJMK'],
    bank_category=['Doprava',]
)
transfers_n_cache = Category(
    name='Переводы и выдача наличных',
    key_words=['Výber', 'Výběr z bankomatu', 'Revolut', 'MMB2713',
               'VÝBĚR HOTOVOSTI'],
    bank_category=['Bankomat']
)


income = Category(
    name='Переводы на счет',
)
unsorted = Category(
    name='Неотсартированные'
)

categories = [food, food_out_side, go_pay, additional, regular_payments, transport, transfers_n_cache]

In [189]:
df['Název kategorie'].isin(cat.bank_category)

0     False
1     False
3     False
4     False
7     False
9     False
10    False
11    False
12    False
13    False
20    False
22    False
23    False
24    False
25    False
26    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
45    False
46    False
48    False
49    False
50    False
51    False
52    False
53    False
54    False
55    False
56    False
57    False
58    False
60    False
61    False
62    False
64    False
65    False
66    False
67    False
68    False
69    False
70    False
71    False
72    False
73    False
Name: Název kategorie, dtype: bool

In [194]:
df = pd.read_csv(data_dir / '12-2022-moneta.csv', delimiter=';')
columns_2_remove = ['Číslo protiúčtu', 'Banka protiúčtu', 'Název účtu příjemce',
       'Splatnost', 'Variabilní Symbol', 'Specifický Symbol',
       'Konstantní Symbol', 'Zpráva pro příjemce', 'Poznámka pro mě',
       'Název trvalého příkazu', 'Popis platby 2']
df.drop(columns_2_remove, inplace=True, axis=1)

In [195]:
df['Částka'] = df['Částka'].str.replace(',', '.').astype({'Částka': 'float64'}) 
income.transaction = df[df['Částka'] > 0].copy()
df.drop(income.transaction.index, inplace=True)
df.fillna('No value', inplace=True)
for cat in categories:
    cat.transaction = df[df['Popis platby'].str.contains('|'.join(cat.key_words), flags=re.IGNORECASE, regex=True)].copy()
    df.drop(cat.transaction.index, inplace=True)
    values = df[df['Název kategorie'].isin(cat.bank_category)].copy()
    df.drop(values.index, inplace=True)
    pd.concat([cat.transaction, values])
unsorted.transaction = df

In [196]:
unsorted.transaction

,Číslo účtu,IBAN,Odesláno,Částka,Měna,Název kategorie,Typ transakce,Popis platby,Bankovní reference
3,233993963/0600,CZ9306000000000233993963,27.12.2022,-230.0,CZK,Služby,Příkaz k úhradě,OKAMŽITÁ ÚHRADA,0233993963:20221227:00009:221226XSB4157917905
7,233993963/0600,CZ9306000000000233993963,27.12.2022,-90.0,CZK,Domácnost,Použití platební karty,TIGER BRNO CZ,0233993963:20221227:00007:221224V0741021
13,233993963/0600,CZ9306000000000233993963,27.12.2022,-280.0,CZK,Bydlení,Použití platební karty,MOBILNI STANEK 4 PONIKLA CZ,0233993963:20221227:00001:221223V0176275
48,233993963/0600,CZ9306000000000233993963,13.12.2022,-390.0,CZK,Jídlo a pití,Použití platební karty,SUMUP *DOMOVINA BRNO CZ,0233993963:20221213:00006:221212V1113569


In [181]:
additional.transaction

,Číslo účtu,IBAN,Odesláno,Částka,Měna,Název kategorie,Typ transakce,Popis platby,Bankovní reference
1,233993963/0600,CZ9306000000000233993963,29.12.2022,-132.70,CZK,Krása a zdraví,Použití platební karty,ROSSMANN VAM DEKUJE BRNO CZ,0233993963:20221229:00002:221228V0788485
20,233993963/0600,CZ9306000000000233993963,21.12.2022,-238.90,CZK,Krása a zdraví,Použití platební karty,ROSSMANN VAM DEKUJE BRNO CZ,0233993963:20221221:00002:221220V0962849
26,233993963/0600,CZ9306000000000233993963,20.12.2022,-329.00,CZK,Zábava,Použití platební karty,SUNSET CINEMA PRAHA CZ,0233993963:20221220:00001:221219V0116819
42,233993963/0600,CZ9306000000000233993963,13.12.2022,-1601.00,CZK,Oblečení,Použití platební karty,PRIMARK BRNO OLYMPIA CZECHIA C,0233993963:20221213:00012:221212V2051149
45,233993963/0600,CZ9306000000000233993963,13.12.2022,-254.00,CZK,Jídlo a pití,Použití platební karty,SB Olympia Brno Brno Modrice C,0233993963:20221213:00009:221212V1533587
46,233993963/0600,CZ9306000000000233993963,13.12.2022,-4355.50,CZK,Oblečení,Použití platební karty,CELIO OLYMPIA BRNO MODRICE CZ,0233993963:20221213:00008:221212V1456311
49,233993963/0600,CZ9306000000000233993963,13.12.2022,-319.00,CZK,Elektronika,Použití platební karty,DATART BRNO CZ,0233993963:20221213:00005:221212V1051351
50,233993963/0600,CZ9306000000000233993963,13.12.2022,-595.00,CZK,Elektronika,Použití platební karty,DATART.CZ ZLIN CZ,0233993963:20221213:00004:221212V0919831
54,233993963/0600,CZ9306000000000233993963,12.12.2022,-39.00,CZK,Sport&Vybavení,Použití platební karty,HULK GYM Brno CZ,0233993963:20221212:00002:221210V0864309
60,233993963/0600,CZ9306000000000233993963,08.12.2022,-507.04,CZK,Zábava,Použití platební karty,PlayStation Network Hilversum,0233993963:20221208:00001:221207V0128193
